## Machine Learning Approach

## Dataset

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.3 MB/s 
     |████████████████████████████████| 212 kB 41.6 MB/s 
     |████████████████████████████████| 115 kB 42.9 MB/s 
     |████████████████████████████████| 120 kB 47.5 MB/s 
     |████████████████████████████████| 127 kB 41.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset

dataset_name = "amazon_reviews_multi"
marc_en = load_dataset(path=dataset_name, name="en")
train_ds = marc_en["train"]
train_ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 200000
})

In [ ]:
product_category = "home"
def filter_for_product(example, product_category=product_category):
    return example["product_category"] == product_category

In [ ]:
product_dataset = marc_en.filter(filter_for_product)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
product_dataset.set_format("pandas")
train_df = product_dataset["train"][:]
test_df = product_dataset["test"][:]

### Load Relevant Dependencies

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string

lemmatizer=WordNetLemmatizer()
STOPWORDS = set(stopwords.words('english'))

import pandas as pd
import pprint
pp = pprint.PrettyPrinter()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### Apply Pre-processing steps

In [ ]:
# Pre-process words per document
# Stopwords and punctuation Removal
# Lemmatized word representation and lowercasing

def word_preprocessor(word:str):
  """Returns a preprocessed word token"""

  if word not in STOPWORDS: # Stop Words Removal
    if word not in string.punctuation: # Punctuation Removal
      if word != '/><br':

        lemma = lemmatizer.lemmatize(word) # Lemmatization
        lemma = lemma.replace(',','').replace('.','') # Punctuation removal
        token = lemma.lower() # Lower Case
        return token

In [ ]:
%%time
# Preprocess training data 
processed_list = list()

for index, row in train_df.iterrows():
  sentence_list = []
  for word in row['review_body'].split(" "): # Tokenization
    token = word_preprocessor(word)
    if token:
      sentence_list.append(token)
  processed_list.append(' '.join(sentence_list))
train_df['review_body_prep'] = processed_list

CPU times: user 3.21 s, sys: 11 ms, total: 3.22 s
Wall time: 3.24 s


In [ ]:
%%time
# Preprocess testing data
'''Important!!!'''
# Pre-processing steps for Testing data should be same as Training data 

processed_list = list()

for index, row in test_df.iterrows():
  sentence_list = []
  for word in row['review_body'].split(" "):
    token = word_preprocessor(word)
    if token:
      sentence_list.append(token)
  processed_list.append(' '.join(sentence_list))
test_df['review_body_prep']= ' '.join(processed_list)

CPU times: user 87.4 ms, sys: 903 µs, total: 88.3 ms
Wall time: 89.5 ms


### Train a Machine Learning Model for the classification task

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
x_train = train_df['review_body_prep']
y_train = train_df['stars']

x_test = test_df['review_body_prep']
y_test = test_df['stars']

In [ ]:
# Using Tf-idf to convert the review into a Tf-idf vectors
tf = TfidfVectorizer()

In [ ]:
classifier = LogisticRegression()
model_tf = Pipeline([('vectorizer',tf),('classifier',classifier)])
model_tf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

### Testing of the Logic

In [ ]:
y_pred_tf = model_tf.predict(x_test)

### Evaluation of the Model

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred_tf)

1.2181818181818183